In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [4]:
df_train.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
83930,John_Ruddy,28,1,0,Wolverhampton Wanderers,Tottenham Hotspur,0.709551,1.823559,False,0,...,3,2,0.0,-130,94,224,0,2020-03-01T14:00:00Z,1920,1.0
83931,Heurelho da Silva_Gomes,28,1,0,Watford,Liverpool,0.477358,2.450632,True,0,...,0,3,0.0,-43,85,128,0,2020-02-29T17:30:00Z,1920,1.0
83932,Joseph_Anang,28,1,0,West Ham United,Southampton,0.759387,0.453058,True,0,...,1,3,0.0,-140,271,411,0,2020-02-29T15:00:00Z,1920,1.0
83933,Erik_Pieters,28,2,0,Burnley,Newcastle United,0.403119,0.641139,False,0,...,0,0,0.0,-6441,1056,7497,0,2020-02-29T15:00:00Z,1920,1.0
83934,Japhet_Tanganga,28,2,75,Tottenham Hotspur,Wolverhampton Wanderers,1.823559,0.709551,True,1,...,3,2,0.0,1798,12124,10326,0,2020-03-01T14:00:00Z,1920,1.0


In [5]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

28

In [5]:
# last_gw = 28

In [6]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [7]:
df_latest
#df_latest[df_latest['player'] == 'Kevin_De Bruyne']

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Chris_Mepham,28,2,0,Bournemouth,Chelsea,0.705844,1.746436,True,0,...,47415,2,2,0.0,-693,10,703,0,2020-02-29T15:00:00Z,1920
1,Paulo_Gazzaniga,28,1,90,Tottenham Hotspur,Wolverhampton Wanderers,1.823559,0.709551,True,1,...,150531,3,2,0.0,-17384,1146,18530,0,2020-03-01T14:00:00Z,1920
2,Max_Meyer,28,3,0,Crystal Palace,Brighton and Hove Albion,0.442761,0.467988,False,0,...,5899,1,0,0.0,-3,34,37,0,2020-02-29T12:30:00Z,1920
3,Fabian_Delph,28,2,0,Everton,Manchester United,1.010525,1.649028,True,0,...,13493,1,1,0.0,3,157,154,0,2020-03-01T14:00:00Z,1920
4,James_Justin,28,2,0,Leicester City,Norwich,1.032252,0.299327,False,0,...,7456,0,1,0.0,-26,43,69,0,2020-02-28T20:00:00Z,1920
5,Che_Adams,28,4,0,Southampton,West Ham United,0.453058,0.759387,False,0,...,23677,1,3,0.0,71,343,272,0,2020-02-29T15:00:00Z,1920
6,Adrian_Mariappa,28,2,0,Watford,Liverpool,0.477358,2.450632,True,0,...,24175,0,3,0.0,-1885,208,2093,0,2020-02-29T17:30:00Z,1920
7,Frederico_Rodrigues de Paula Santos,28,3,90,Manchester United,Everton,1.649028,1.010525,False,1,...,25402,1,1,12.0,2465,3935,1470,1,2020-03-01T14:00:00Z,1920
8,Tanguy_Ndombele,28,3,14,Tottenham Hotspur,Wolverhampton Wanderers,1.823559,0.709551,True,1,...,168645,3,2,0.0,-118,657,775,0,2020-03-01T14:00:00Z,1920
9,Andy_Lonergan,28,1,0,Liverpool,Watford,2.450632,0.477358,False,0,...,1859,0,3,0.0,-3,6,9,0,2020-02-29T17:30:00Z,1920


In [8]:
df_latest.shape

(504, 34)

In [9]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [10]:
last_play_proba.head()

,player,play_proba
32,Steve_Cook,1.0
33,Nathan_Aké,1.0
34,Charlie_Daniels,0.0
35,Chris_Mepham,0.0
36,Simon_Francis,1.0


In [11]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [12]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
0,Chris_Mepham,28,2,0,Bournemouth,Chelsea,0.705844,1.746436,True,0,...,2,2,0.0,-693,10,703,0,2020-02-29T15:00:00Z,1920,0.00
1,Paulo_Gazzaniga,28,1,90,Tottenham Hotspur,Wolverhampton Wanderers,1.823559,0.709551,True,1,...,3,2,0.0,-17384,1146,18530,0,2020-03-01T14:00:00Z,1920,1.00
2,Max_Meyer,28,3,0,Crystal Palace,Brighton and Hove Albion,0.442761,0.467988,False,0,...,1,0,0.0,-3,34,37,0,2020-02-29T12:30:00Z,1920,1.00
3,Fabian_Delph,28,2,0,Everton,Manchester United,1.010525,1.649028,True,0,...,1,1,0.0,3,157,154,0,2020-03-01T14:00:00Z,1920,1.00
4,James_Justin,28,2,0,Leicester City,Norwich,1.032252,0.299327,False,0,...,0,1,0.0,-26,43,69,0,2020-02-28T20:00:00Z,1920,1.00
5,Che_Adams,28,4,0,Southampton,West Ham United,0.453058,0.759387,False,0,...,1,3,0.0,71,343,272,0,2020-02-29T15:00:00Z,1920,1.00
6,Adrian_Mariappa,28,2,0,Watford,Liverpool,0.477358,2.450632,True,0,...,0,3,0.0,-1885,208,2093,0,2020-02-29T17:30:00Z,1920,1.00
7,Frederico_Rodrigues de Paula Santos,28,3,90,Manchester United,Everton,1.649028,1.010525,False,1,...,1,1,12.0,2465,3935,1470,1,2020-03-01T14:00:00Z,1920,1.00
8,Tanguy_Ndombele,28,3,14,Tottenham Hotspur,Wolverhampton Wanderers,1.823559,0.709551,True,1,...,3,2,0.0,-118,657,775,0,2020-03-01T14:00:00Z,1920,1.00
9,Andy_Lonergan,28,1,0,Liverpool,Watford,2.450632,0.477358,False,0,...,0,3,0.0,-3,6,9,0,2020-02-29T17:30:00Z,1920,1.00


In [13]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [14]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
83930,John_Ruddy,28,1,0,Wolverhampton Wanderers,Tottenham Hotspur,0.709551,1.823559,False,0,...,3,2,0.0,-130,94,224,0,2020-03-01T14:00:00Z,1920,1.0
83931,Heurelho da Silva_Gomes,28,1,0,Watford,Liverpool,0.477358,2.450632,True,0,...,0,3,0.0,-43,85,128,0,2020-02-29T17:30:00Z,1920,1.0
83932,Joseph_Anang,28,1,0,West Ham United,Southampton,0.759387,0.453058,True,0,...,1,3,0.0,-140,271,411,0,2020-02-29T15:00:00Z,1920,1.0
83933,Erik_Pieters,28,2,0,Burnley,Newcastle United,0.403119,0.641139,False,0,...,0,0,0.0,-6441,1056,7497,0,2020-02-29T15:00:00Z,1920,1.0
83934,Japhet_Tanganga,28,2,75,Tottenham Hotspur,Wolverhampton Wanderers,1.823559,0.709551,True,1,...,3,2,0.0,1798,12124,10326,0,2020-03-01T14:00:00Z,1920,1.0


In [15]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train.csv')

In [6]:
# df_train_new = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

In [7]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [8]:
fixtures.head(10)

,home_team,away_team,gw
270,Aston Villa,Sheffield United,34
274,Manchester City,Arsenal,29
280,Arsenal,West Ham United,29
281,Burnley,Tottenham Hotspur,29
282,Chelsea,Everton,29
283,Crystal Palace,Watford,29
284,Leicester City,Aston Villa,29
285,Liverpool,Bournemouth,29
286,Manchester United,Manchester City,29
287,Sheffield United,Norwich,29


In [9]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [10]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,34,Aston Villa,Sheffield United,7,49
1,29,Manchester City,Arsenal,43,3
2,29,Arsenal,West Ham United,3,21
3,29,Burnley,Tottenham Hotspur,90,6
4,29,Chelsea,Everton,8,11


In [11]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [12]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [13]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,34,Sheffield United,Aston Villa,Lys_Mousset,4,46,100,False
1,34,Sheffield United,Aston Villa,Enda_Stevens,2,51,100,False
2,34,Sheffield United,Aston Villa,Kieron_Freeman,2,42,None,False
3,34,Sheffield United,Aston Villa,Jack_O'Connell,2,47,None,False
4,34,Sheffield United,Aston Villa,George_Baldock,2,51,None,False


In [14]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,34,Aston Villa,Sheffield United,Ahmed_El Mohamady,2,42,None,True
1,34,Aston Villa,Sheffield United,James_Chester,2,43,0,True
2,34,Aston Villa,Sheffield United,Neil_Taylor,2,43,None,True
3,34,Aston Villa,Sheffield United,Kortney_Hause,2,44,100,True
4,34,Aston Villa,Sheffield United,Jonathan_Kodjia,4,54,0,True


In [15]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 9.0,
 'Aaron_Cresswell': 90.0,
 'Aaron_Lennon': 1.6,
 'Aaron_Mooy': 86.2,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 90.0,
 'Abd-Al-Ali Morakinyo Olaposi_Koiki': 0.0,
 'Abdoulaye_Doucouré': 87.4,
 'Adam_Idah': 0.0,
 'Adam_Lallana': 4.142857142857143,
 'Adam_Masina': 90.0,
 'Adam_Smith': 90.0,
 'Adam_Webster': 72.0,
 'Adama_Traoré': 64.0,
 'Adrian_Mariappa': 54.0,
 'Adrián_San Miguel del Castillo': 0.0,
 'Ahmed_El Mohamady': 0.0,
 'Ainsley_Maitland-Niles': 0.0,
 'Albian_Ajeti': 0.0,
 'Alex_Iwobi': 30.75,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 66.75,
 'Alexander_Tettey': 85.8,
 'Alexandre_Lacazette': 45.75,
 'Alexis_Mac Allister': 0.0,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 17.6,
 'Alisson_Ramses Becker': 90.0,
 'Allan_Saint-Maximin': 54.0,
 'Andre_Gray': 1.8,
 'Andreas_Christensen': 81.0,
 'Andreas_Pereira': 25.2,
 'Andrew_Robertson': 90.0,
 'Andrew_Surman': 35.4,
 'Andros_Townsend': 3.25,
 'André Filipe_Tavares Gomes': 56.0,
 'Andy_Lonergan'

In [16]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [17]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,34,Aston Villa,Sheffield United,Ahmed_El Mohamady,2,4.2,1.00,True,0.538315,0.294179,1920,0.000000
1,34,Aston Villa,Sheffield United,James_Chester,2,4.3,0.00,True,0.538315,0.294179,1920,0.000000
2,34,Aston Villa,Sheffield United,Neil_Taylor,2,4.3,1.00,True,0.538315,0.294179,1920,0.000000
3,34,Aston Villa,Sheffield United,Kortney_Hause,2,4.4,1.00,True,0.538315,0.294179,1920,90.000000
4,34,Aston Villa,Sheffield United,Jonathan_Kodjia,4,5.4,0.00,True,0.538315,0.294179,1920,0.000000
5,34,Aston Villa,Sheffield United,Ørjan_Nyland,1,4.3,1.00,True,0.538315,0.294179,1920,0.000000
6,34,Aston Villa,Sheffield United,Conor_Hourihane,3,5.7,1.00,True,0.538315,0.294179,1920,7.750000
7,34,Aston Villa,Sheffield United,Jack_Grealish,3,6.6,1.00,True,0.538315,0.294179,1920,90.000000
8,34,Aston Villa,Sheffield United,Anwar_El Ghazi,3,5.5,1.00,True,0.538315,0.294179,1920,60.750000
9,34,Aston Villa,Sheffield United,John_McGinn,3,5.4,0.00,True,0.538315,0.294179,1920,0.000000


In [18]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')